In [3]:
import json
from pathlib import Path
import tensorflow as tf
import src.data.build_dataset as build_dataset
import src.data.input_pipeline as input_pipeline
import importlib
import src.models.transformer.metrics as metrics
import src.models.transformer.utils as utils

# importlib.reload(metrics)

# acc =tf.metrics.CategoricalAccuracy()
# pred = tf.random.uniform(shape=(1,2,25))
# true = tf.random.uniform(shape=(1,2,25))
# pred_pos = tf.constant([[[0]]])
# true_pos = tf.constant([1])
# categories = tf.constant([[1,2]])
# metrics.fitb_acc(pred,true,pred_pos,true_pos, categories, acc)
# print(acc.result())




file = [r"D:\David\Škola\outfit-generation\data\processed\tfrecords\fitb-valid-neg.tfrecord"]
importlib.reload(input_pipeline)
lookup = utils.build_category_lookup_table()
dataset = input_pipeline.get_fitb_dataset(file, True, lookup).take(50)


for x in dataset:
    inputs, input_categories, targets, target_categories, target_position = x
    print(target_categories)

tf.Tensor([1 1 1 2], shape=(4,), dtype=int64)
tf.Tensor([9 9 3 1], shape=(4,), dtype=int64)
tf.Tensor([4 7 2 2], shape=(4,), dtype=int64)
tf.Tensor([4 1 7 7], shape=(4,), dtype=int64)
tf.Tensor([7 8 1 1], shape=(4,), dtype=int64)
tf.Tensor([3 9 9 4], shape=(4,), dtype=int64)
tf.Tensor([9 3 3 4], shape=(4,), dtype=int64)
tf.Tensor([9 7 1 9], shape=(4,), dtype=int64)
tf.Tensor([9 4 3 3], shape=(4,), dtype=int64)
tf.Tensor([2 6 6 7], shape=(4,), dtype=int64)
tf.Tensor([5 4 9 9], shape=(4,), dtype=int64)
tf.Tensor([9 5 4 9], shape=(4,), dtype=int64)
tf.Tensor([9 5 9 4], shape=(4,), dtype=int64)
tf.Tensor([7 4 5 9], shape=(4,), dtype=int64)
tf.Tensor([3 4 4 4], shape=(4,), dtype=int64)
tf.Tensor([8 1 6 3], shape=(4,), dtype=int64)
tf.Tensor([2 3 9 2], shape=(4,), dtype=int64)
tf.Tensor([4 9 4 9], shape=(4,), dtype=int64)
tf.Tensor([6 6 4 9], shape=(4,), dtype=int64)
tf.Tensor([4 6 3 3], shape=(4,), dtype=int64)
tf.Tensor([7 1 3 8], shape=(4,), dtype=int64)
tf.Tensor([9 2 8 2], shape=(4,), d